In [136]:
import io
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt
#from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import preprocessing
from sklearn import metrics
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [160]:
column_rating = ['userId', 'movieId', 'rating']
rating_score = pd.read_csv('C:/Users/cajethan/Desktop/INFO_MGT/thesis_info_mgt/datasets/archive/ratings_small.csv', usecols = column_rating)
rating_score['movieId'] = rating_score['movieId'].astype('str')
rating_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 3 columns):
userId     100004 non-null int64
movieId    100004 non-null object
rating     100004 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ MB


In [161]:
movie_titles = ['id', 'original_title']
titles = pd.read_csv('C:/Users/cajethan/Desktop/INFO_MGT/thesis_info_mgt/datasets/archive/movies_metadata.csv', usecols = movie_titles)
titles = titles.rename(columns = {"id": "movieId","original_title":"title"})
titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 2 columns):
movieId    45466 non-null object
title      45466 non-null object
dtypes: object(2)
memory usage: 710.5+ KB


In [162]:
merged_data = pd.merge(rating_score, titles)
merged_data.head()

,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
1,4,1371,4.0,Rocky III
2,7,1371,3.0,Rocky III
3,19,1371,4.0,Rocky III
4,21,1371,3.0,Rocky III


In [163]:
def assign_to_testset(df):
    sampled_ids = np.random.choice(df.index,size=np.int64(np.ceil(df.index.size * 0.2)),replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df

merged_data['for_testing'] = False
grouped = merged_data.groupby('userId', group_keys=False).apply(assign_to_testset)
train_data = merged_data[grouped.for_testing == False]
test_data = merged_data[grouped.for_testing == True]
print(train_data.shape)
print(test_data.shape)
print(train_data.index & test_data.index)

print("Training data_set has "+ str(train_data.shape[0]) +" ratings")
print("Test data set has "+ str(test_data.shape[0]) +" ratings")

(35731, 5)
(9263, 5)
Int64Index([], dtype='int64')
Training data_set has 35731 ratings
Test data set has 9263 ratings


In [141]:
#compute the mean 
mean_rating = train_data.groupby(by = 'userId', as_index = False)['rating'].mean()
#find the average 
avg_rating = pd.merge(train_data, mean_rating, on = 'userId')
avg_rating.head()

,userId,movieId,rating_x,title,for_testing,rating_y
0,7,1371,3.0,Rocky III,False,3.333333
1,7,1405,5.0,Greed,False,3.333333
2,7,272,3.0,Batman Begins,False,3.333333
3,7,364,3.0,Batman Returns,False,3.333333
4,7,377,3.0,A Nightmare on Elm Street,False,3.333333


In [142]:
#rename rating_x and rating_y
avg_rating = avg_rating.rename(columns = {"rating_x": "rating", "rating_y": "mean_rating"})
avg_rating = avg_rating.drop(columns = "for_testing")
avg_rating.head()

,userId,movieId,rating,title,mean_rating
0,7,1371,3.0,Rocky III,3.333333
1,7,1405,5.0,Greed,3.333333
2,7,272,3.0,Batman Begins,3.333333
3,7,364,3.0,Batman Returns,3.333333
4,7,377,3.0,A Nightmare on Elm Street,3.333333


In [143]:
#now we get the adjusted rating
avg_rating['base_rating'] = avg_rating['rating'] - avg_rating['mean_rating']
avg_rating.head()

,userId,movieId,rating,title,mean_rating,base_rating
0,7,1371,3.0,Rocky III,3.333333,-0.333333
1,7,1405,5.0,Greed,3.333333,1.666667
2,7,272,3.0,Batman Begins,3.333333,-0.333333
3,7,364,3.0,Batman Returns,3.333333,-0.333333
4,7,377,3.0,A Nightmare on Elm Street,3.333333,-0.333333


In [164]:
avg_rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35731 entries, 0 to 35730
Data columns (total 6 columns):
userId         35731 non-null int64
movieId        35731 non-null object
rating         35731 non-null float64
title          35731 non-null object
mean_rating    35731 non-null float64
base_rating    35731 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.9+ MB


In [173]:
#make a pivot_table out of the dataframe
final = pd.pivot_table(avg_rating, values = 'base_rating', index = ['userId'], columns = ['movieId'])
final.head()

movieId,100,100017,100272,100450,101,101362,1018,102,1023,1024,...,98491,986,987,988,99,990,991,992,994,996
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
#replace all the NaN values with the movie average in each row
df_final = final.fillna(final.mean(axis = 0))
df_final.head()

movieId,100,100017,100272,100450,101,101362,1018,102,1023,1024,...,98491,986,987,988,99,990,991,992,994,996
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.182314,-1.162162,0.325758,0.185714,0.510056,-0.547037,-0.215644,-0.573438,0.592053,-0.33001,...,0.741974,-0.200314,-0.339943,0.204393,0.001216,-0.713816,-0.066232,-0.723441,0.710257,0.026719
2,-0.182314,-1.162162,0.325758,0.185714,0.510056,-0.547037,-0.215644,-0.573438,0.592053,-0.33001,...,0.741974,-0.200314,-0.339943,0.204393,0.001216,-0.713816,-0.066232,-0.723441,0.710257,0.026719
3,-0.182314,-1.162162,0.325758,0.185714,0.510056,-0.547037,-0.215644,-0.573438,0.592053,-0.33001,...,0.741974,-0.200314,-0.339943,0.204393,0.001216,-0.713816,-0.066232,-0.723441,0.710257,0.026719
4,-0.182314,-1.162162,0.325758,0.185714,0.510056,-0.547037,-0.215644,-0.573438,0.592053,-0.33001,...,0.741974,-0.200314,-0.339943,0.204393,0.001216,-0.713816,-0.066232,-0.723441,0.710257,0.026719
5,-0.182314,-1.162162,0.325758,0.185714,0.510056,-0.547037,-0.215644,-0.573438,0.592053,-0.33001,...,0.741974,-0.200314,-0.339943,0.204393,0.001216,-0.713816,-0.066232,-0.723441,0.710257,0.026719


In [175]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 671 entries, 1 to 671
Columns: 2672 entries, 100 to 996
dtypes: float64(2672)
memory usage: 13.7 MB


In [176]:
print(df_final.dtypes)

movieId
100       float64
100017    float64
100272    float64
100450    float64
101       float64
           ...   
990       float64
991       float64
992       float64
994       float64
996       float64
Length: 2672, dtype: object


In [177]:
# common_ratings = lambda x, y : np.argwhere(
#     (~np.isnan(x)) & (~np.isnan(y)) & (x >= 0) & (y >= 0)
# )[:, 0]

# def common_cosine(v1, v2):
#     common = common_ratings(v1, v2)
#     v1c, v2c = v1[common], v2[common]
#     if len(v1c) <= 1: return 0
#     return 1 - cosine(v1c, v2c)

In [178]:
#now compute the pairwise_distances between the ratings
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist
#from scipy.spatial.distance import jaccard

# distances = pdist(df_final, 'cosine')
# distances

cos_sim = cosine_similarity(df_final)
np.fill_diagonal(cos_sim, 0)
#create an empty dataframe to store all the similar movies 
sim_movies = pd.DataFrame(cos_sim, index = df_final.index, columns = df_final.index)
sim_movies.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.985227,0.996327,0.983160,0.993267,0.992725,0.991548,0.989842,0.996138,0.996669,...,0.992140,0.999637,0.989757,0.961021,0.993472,0.991405,0.997953,0.998973,0.992024,0.994163
2,0.985227,0.000000,0.982512,0.968079,0.979338,0.978685,0.979377,0.975687,0.982257,0.982079,...,0.978253,0.985412,0.976006,0.941949,0.977896,0.972641,0.983560,0.984377,0.975606,0.980223
3,0.996327,0.982512,0.000000,0.980037,0.990482,0.989704,0.990168,0.986454,0.992781,0.993610,...,0.987731,0.996594,0.986321,0.957799,0.991850,0.989330,0.994673,0.996500,0.988186,0.990868
4,0.983160,0.968079,0.980037,0.000000,0.977132,0.977084,0.977212,0.975033,0.979818,0.980538,...,0.976407,0.983491,0.973595,0.950143,0.977506,0.975642,0.981792,0.983206,0.976357,0.978018
5,0.993267,0.979338,0.990482,0.977132,0.000000,0.986620,0.985899,0.983344,0.990061,0.991209,...,0.986677,0.993480,0.984622,0.956711,0.987834,0.984812,0.991805,0.992822,0.985332,0.988218


In [179]:
#HOW DO WE DETERMINE K???

In [180]:
def find_neighbors(df, n):
    sort = np.argsort(df.values, axis = 1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending = False)
            .iloc[:n].index,
            index = ['top{}'.format(i) for i in range(1, n + 1)]), axis = 1)
    return df

In [181]:
#top n similar neighbors for each user
similar_users = find_neighbors(sim_movies, 30)
similar_users.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
1,663,448,485,54,314,566,129,601,229,443,...,438,76,377,171,374,181,11,104,622,532
2,314,448,241,622,663,449,485,54,566,1,...,464,249,335,357,206,71,29,267,532,347
3,448,314,663,669,54,485,1,566,347,129,...,71,29,171,503,40,635,280,446,141,374
4,576,485,663,448,601,54,314,566,129,229,...,335,171,181,377,40,167,374,438,11,532
5,229,347,489,663,448,485,54,1,314,566,...,29,526,669,40,13,167,438,76,567,181


In [182]:
#next, we return all the similar movies
def similarities(user1, user2):
    common_movies = avg_rating[avg_rating.userId == user1].merge(avg_rating[avg_rating.userId == user2]
                    ,on = 'movieId'
                    ,how = "inner")
    return common_movies

In [183]:
sim_test = similarities(20, 400)
sim_test = sim_test.loc[:, ['rating_x', 'rating_y', 'title']].rename(columns = {"rating_x": "user1_rating", "rating_y": "user2_rating"})
sim_test.head()

C:\Users\cajethan\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,user1_rating,user2_rating,title
0,3.0,4.0,NaN
1,4.5,4.0,NaN
2,3.0,3.0,NaN
3,2.5,4.0,NaN
4,2.0,4.0,NaN


In [196]:
def predict_score (user, item):
    #i = float(item)
    user_avg_rating = avg_rating.loc[avg_rating['userId'] == user, 'mean_rating'].values[0]
    sum_of_k_similar_users = similar_users[similar_users.index==user].values.squeeze().tolist()
    rated_item = df_final.loc[:, item].index.isin(sum_of_k_similar_users).squeeze().tolist()
    corr = sim_movies.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score



# def User_item_score(user,item):
#     #.The values() method returns a view object that displays a list of all the values in the dictionary. 
#     a = similar_users[similar_users.index==user].values
#     #squeeze() removes single-dimensional entries - it removes the user's similarity with itself
#     #tolist() makes a list of values (userid's  in this instance)
#     b = a.squeeze().tolist()
    
#     c = df_final.loc[:, item]
#     d = c[c.index.isin(b)]
#     f = d[d.notnull()]
#     # get the average rating of the target user
#     avg_user = mean_rating.loc[mean_rating['userId'] == user,'rating'].values[0]
#     index = f.index.values.squeeze().tolist()
#     corr = sim_movies.loc[user,index]
#     fin = pd.concat([f, corr], axis=1)
#     fin.columns = ['adg_score','correlation']
#     fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
#     nume = fin['score'].sum()
#     deno = fin['correlation'].sum()
#     final_score = avg_user + (nume/deno)
#     return final_score

In [198]:
score = predict_score(20, 100017)
print("score (u,i) is", score)

KeyError: 'Iron Man'

In [ ]:
# #extract the userIds to a list
# user_array = avg_rating["userId"].tolist()
# user_array

In [ ]:
# user_to_recommend_for = user_array.index(7)
# user_to_recommend_for

In [ ]:
# np.version.version